In [3]:
import openai

In [4]:
import os
from openai import AzureOpenAI

In [5]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

True

In [6]:
client = AzureOpenAI(
    api_key=os.getenv('aikey'),
    api_version="2024-02-01",
    azure_endpoint = "https://ragprojectv1.openai.azure.com/"
)

gpt_four = "gpt-four-oai"

In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-21 12:34:18--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-21 12:34:19 (32.3 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import minsearch

In [11]:
import json

In [13]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [14]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [15]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [16]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [17]:
index.fit(documents)

In [27]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=2
    )

    return results

In [28]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [29]:
def llm(prompt):
    response = client.chat.completions.create(
        model= gpt_four, # model = "deployment_name".
        messages=[
            {"role": "system", "content": "Act as a standup comdeian"},
            {"role": "user", "content": prompt}
        ], max_tokens= 100, temperature= 0
    )
    
    print(response.choices[0].message.content)

In [30]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [31]:
rag('the course has already started, can I still enroll?')

Yes, you can still enroll in the course even after it has started. You are eligible to submit the homeworks, but keep in mind that there will be deadlines for turning in the final projects. So, make sure not to leave everything for the last minute!
